In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('novel_story_sep.csv', lineterminator='\n')

In [4]:
df['story_sep']

0       어디 속 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 작가 이민 진의 장편소설 파...
1       어디 속 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 작가 이민 진의 장편소설 파...
2       자꾸 편의점 살 오늘 위로 편의점 밤 정체 불명 알바로 시작 웃음 감동 나비효과 망...
3         토끼 머리 손가락 몸 안녕 내 사랑 덫 흉터 나 집 바람 모래 지배자 재회 작가 말 
4       괴물 내 또 다른 괴물 영화 사건 매혹 문체 시선 한국 영 어덜트 소설 아몬드 타인...
                              ...                        
1201    기간 사랑 온 한국 대표 판타지 소설 윤 현승 작가 대표 작 늑대 최신 개정판 늑대...
1202    불행 날 아이 운명 거부 신비 도시 버 무어 제시카 타운센드 무려 시간 동안 공 이...
1203    아사히 신문 지난 천 년 일본 문학자 투표 위 무라카미 하루키 강상중 가장 작가 나...
1204    인류 기원 상상 인류 미래 백만 년 가니메데 거인 인류 최초 지적 생명체 거인 종족...
1205    경제 정책 하나 사회 파급 나비효과 시안 생각 비판 얼마나 대하 생각 소설 경제학 ...
Name: story_sep, Length: 1206, dtype: object

https://happy-obok.tistory.com/5

### LDA 모델에 들어갈 객체(dictionary, corpus) 만들고 학습하기

0. gensim 라이브러리 설치 : 자연어를 벡터로 변환하는데 필요한 대부분의 편의 기능을 제공하고 있는 라이브러리 (Word2vec 포함)

In [ ]:
# !pip install gensim==3.8.3

In [1]:
import gensim
gensim.__version__

'3.8.3'

1. gensim.models.wrappers.LdaMallet 모듈을 사용해서 LDA 모델 개수 추정
- https://radimrehurek.com/gensim_3.8.3/models/wrappers/ldamallet.html

In [5]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.wrappers import LdaMallet

1) LDA 모델에 들어갈 객체를 만든다.
- id2word : dictionary 에 list of list of str 형식의 documents를 입력하면 Dictionary가 학습됨. 전체 말뭉치에 단어가 겹치지 않도록 하나씩 dictionary에 저장됨.
- corpus : 단어들을 bag-of-words 형태에서 list if (token_id, token_count) 2-tuples로 변환한다.


= [[(id2 word [id], freq) for id, freq in cp] for cp in corpus [:50]]

In [6]:
stories = df['story_sep']

In [7]:
data_word = []
for story in stories:
    data = list(str(story).split())
    data_word.append(data)

In [8]:
id2word = corpora.Dictionary(data_word)

In [9]:
id2word.filter_extremes(no_below = 5) # 5회 이하로 등장한 단어는 삭제
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

mallet_path = '../감성사전/mallet-2.0.8/bin/mallet' 
# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=9, id2word=id2word)

---

### 토픽 수 별로 일관성 점수를 계산해서 가장 좋은 토픽 수의 모델 찾기

models.coherencemodel 주제 모델에 대한 주제 일관성을 계산

* model :주제가 제공되지 않은 경우 사전 훈련된 주제 모델을 제공해야 합니다. 현재 지원 LdaModel, LdaMulticore, LdaMallet와 LdaVowpalWabbit.
* topics(list of list of str, optional) :토큰 화 된 토픽의 목록
* texts (list of list of str, optional) :슬라이딩 창 기반 (예 : coherence =c_something) 확률 추정 기를 사용하는 일관성 모델에 필요한 토큰 화 된 텍스트.
* corpus (iterable of list of (int, number), optional) :BoW 형식의 코퍼스.
* dictionary (Dictionary, optional) : Gensim dictionary mapping of id word to create corpus. If model.id2 word is present, this is not needed. If both are provided, passed dictionary will be used.
* coherence ({'u_mass', 'c_v', 'c_uci', 'c_npmi'}, optional) :
* topn (int, optional) : 각 주제에서 추출할 최상위 단어 수에 해당하는 정수
* processes (int, optional) : 확률 추정 단계에 사용할 프로세스 수

#### <Coherence 점수를 계산하여 좋은 LDA 모델 찾기>

In [ ]:
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# coherence_ldamallet #응집성 지수

In [10]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [11]:
# Can take a long time to run.
limit=11; start=7; step=1;
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=start, limit=limit, step=step)

Mallet LDA: 7 topics, 3 topic bits, 111 topic mask
Data loaded.
max tokens: 1757
total tokens: 308503
<10> LL/token: -9.0412
<20> LL/token: -8.61246
<30> LL/token: -8.47564
<40> LL/token: -8.40695

0	7.14286	시대 우리 한국 문학 역사 위 사회 장 책 가장 세기 이자 전쟁 간 대표 은 자유 태백산맥 통해 제 
1	7.14286	아이 청소년 친구 엄마 위해 우리 더 주인공 제 학교 너 가족 일 세 시작 날 아버지 몸 대한 자기 
2	7.14286	소녀 권 책 세상 일 온 시작 속 드라마 왕 사랑 마음 다시 은 알 를 살 내 향 김 
3	7.14286	알 그녀 사랑 이름 를 생각 독자 내 나 누군가 저 남편 시간 과정 왜 모든 눈 위 발견 네 
4	7.14286	독자 등 책 사건 의 간 전 은 통해 가장 문학 미스터리 작 시리즈 선정 베스트셀러 일본 부 상 세계 
5	7.14286	인간 의 여성 세계 독자 위해 대한 사회 주인공 등장 화 현실 속 인류 중 등 사실 무엇 삶 상황 
6	7.14286	사랑 삶 우리 나 마음 때 생각 집 순간 죽음 시간 두 속 일 서로 그녀 소년 세계 아내 은 

<50> LL/token: -8.37006
<60> LL/token: -8.34932
<70> LL/token: -8.3339
<80> LL/token: -8.3208
<90> LL/token: -8.31374

0	7.14286	우리 문학 시대 역사 한국 위 장 사회 책 가장 세기 이자 은 간 전쟁 대표 자유 등 태백산맥 다시 
1	7.14286	아이 청소년 엄마 친구 가족 우리 너 학교 위해 더 날 주인공 인물 아버지 세 일 제 시작 대한 삶 
2	7.14286	소녀 권 책 온 속 세상 일 시작 드라마 사랑 마음 왕 를 다시 은 살 알 그녀 위해 내 
3	7.14286	알 나 생각 이름 그녀 내 시간 사랑 모든 

In [ ]:
x = range(start, limit, step)
topic_num = 0
count = 0
max_coherence = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", cv)
    coherence = cv
    if coherence >= max_coherence:
        max_coherence = coherence
        topic_num = m
        model_list_num = count   
    count = count+1

In [ ]:
# Select the model and print the topics
optimal_model = model_list[model_list_num]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    # sent_topics_df = pd.concat([sent_topics_df, Data['text'],Data['timestamp'],Data['tweet_url'],Data['screen_name'],Data['label']], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts)

In [ ]:
# Format
df_topic_tweet = df_topic_sents_keywords.reset_index()
df_topic_tweet.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text','Timestamp', 'Tweet_url','Screen_name','label']

# Show각 문서에 대한 토픽
#df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])
#df_topic_tweet

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [ ]:
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", "Text", "timestamp", "tweet_url","screen_name","label","Num_Documents", "Perc_Documents"]
lda_inform = lda_inform[["Topic_Num","Keywords","Num_Documents","Perc_Documents"]]
lda_inform
#lda_inform.Topic_Num = lda_inform.Topic_Num.astype(int)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'] +1
df_topic_tweet.Dominant_Topic = df_topic_tweet.Dominant_Topic.astype(str)
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'].str.split('.').str[0]